In [ ]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index
!pip install -q pypdf
!pip install -q tiktoken==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 r

In [ ]:
import os
import json
from google.colab import userdata

os.environ['GRADIENT_ACCESS_TOKEN'] = 'VBMMplZGvWIAsixfyJRFhtOIqeXEyaB9'
os.environ['GRADIENT_WORKSPACE_ID'] =  '4cd01bd1-3483-42a1-957e-31aabd989166_workspace'

In [ ]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings import GradientEmbedding
from llama_index.llms import GradientBaseModelLLM
from llama_index.vector_stores import CassandraVectorStore

In [ ]:
import cassandra
print (cassandra.__version__)

3.29.0


DB


In [ ]:
# This secure connect bundle is autogenerated when you donwload your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-docqa-db.zip'
}

# This token json file is autogenerated when you donwload your token,
# if yours is different update the file name below
with open("poojitha.kathi29@gmail.com-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(137188156033296) d4f5d2bc-40f0-48f7-87d0-804777133922-us-east1.db.astra.datastax.com:29042:81ac5d58-d8e9-4fa4-abdc-c5441a1108ba> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


4.0.11-c44bc828c0c8


In [ ]:
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=500,
)

In [ ]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

In [ ]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=256,
)

set_global_service_context(service_context)

In [ ]:

documents = SimpleDirectoryReader("/content/Documents").load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 266 document(s).


In [ ]:

index = VectorStoreIndex.from_documents(documents,
                                        service_context=service_context)
query_engine = index.as_query_engine()


In [ ]:
response = query_engine.query("What is the system architecture of AURIX™ TC3xx Platform?")
print(response)

Based on the provided context information, the system architecture of the AURIX™ TC3xx Platform is:

The AURIX™ TC3xx Platform is a family of high-performance microcontrollers with multiple TriCore CPUs, program and data memories, buses, bus arbitration, interrupt system, DMA controller, and a powerful set of on-chip peripherals. The platform is designed to meet the needs of the most demanding embedded control systems applications where the competing issues of price/performance, real-time responsiveness, computational power, data bandwidth, and power consumption are key design elements.

The system architecture of the AURIX™ TC3xx Platform consists of:

1. TriCore CPUs: The platform uses multiple TriCore CPUs, which are high-performance processors that combine RISC (Reduced Instruction Set Computing) processor architecture, DSP (Digital Signal Processing) operations and addressing modes, and on-chip memories and peripherals.
2. System Peripheral Bus (SPB): The TriCore CPUs and system p

In [ ]:
response = query_engine.query("What is multi head attention")
print(response)

Based on the context information provided, multi-head attention refers to a mechanism in the arbitration algorithm that allows multiple Master IDs to be considered in each arbitration cycle.

In the algorithm, the lowest MCI ID number in the group that is requesting is selected first, followed by the next higher MCI ID number and so on. This means that each Master ID has a chance to be selected in each arbitration cycle, regardless of whether other Masters are requesting in the same cycle.

By considering multiple Master IDs in each cycle, the algorithm can ensure that each Master has a fair chance of being selected, even if there are other Masters requesting in the same cycle. This is particularly useful in situations where only one Master is requesting over a given time period, as that Master will be selected consecutively during multiple arbitration rounds without incurring any latency.

In summary, multi-head attention in this context refers to the mechanism that allows the algorit

In [ ]:
response = query_engine.query("What is local memory unit?")
print(response)

Based on the context information provided, the Local Memory Unit (LMU) is a unit that provides a small amount of SRAM memory that is accessible only to the CPU. The LMU is connected to the CPU through the DLMU (CPU DLMU or LMU LMURAM) and DAM (DAMRAM) interfaces. The LMU is used to store temporary data and instructions that are being processed by the CPU. It has a total capacity of 64 Kbyte.


In [ ]:
response = query_engine.query("What is AURIX™ TC3xx Platform Firmwar")
print(response)

Based on the provided context information, the answer to the query "What is AURIX™ TC3xx Platform Firmware?" is:

AURIX™ TC3xx Platform Firmware refers to the software that is stored in the non-volatile memory of the AURIX™ TC3xx microcontroller and is used to control its operation. This firmware includes the low-level software that interfaces with the hardware components of the microcontroller, such as the TriCore CPUs, program and data memories, buses, bus arbitration, interrupt system, DMA controller, and on-chip peripherals. The firmware also includes the operating system, device drivers, and application software that run on the microcontroller.

In other words, the AURIX™ TC3xx Platform Firmware is the software that enables the microcontroller to perform its intended functions and interact with the external world through its on-chip peripherals and buses.


In [ ]:
response = query_engine.query("Where all data loading phase present in the document")
print(response)

Based on the additional context provided, the data loading phase is present in the following locations in the document:

1. Page 501C: 0000H - 501C 2FFFH - C P U 2 P r o g r a m C a c h e T A G S R A M1)
2. Page 6000: 0000H - 6003 BFFFH - 240 Kbyte CPU1 Data Scratch-Pad SRAM (CPU1 DSPR)Access Access

Therefore, the data loading phase is present in these two locations in the document.

Note that the original answer provided earlier was correct, and these additional locations do not provide any new information about the data loading phase.


In [ ]:
response = query_engine.query("explain Power Supply Friendly Debug Monitor")
print(response)

Based on the provided context information, the Power Supply Friendly Debug Monitor (PSFDM) is a routine inside the AURIX™ TC3xx Platform BootROM that is designed to minimize the risk of voltage over/undershoot in the system during debugging.

The PSFDM routine is an independent module that is not executed during device startup. Its purpose is to monitor the power supply voltage of the system and take appropriate actions to avoid voltage over/undershoot when more than one CPU is halted by the On-Chip Debug System (OCDS). This is done by:

1. Minimizing the risk of getting EVR voltage over/undershoot due to a sudden current drop when the CPUs are halted.
2. Avoiding a current peak when the CPUs are released from halt.

To achieve this, the PSFDM routine uses several techniques, including:

1. Independent/standalone module: The PSFDM routine is not executed during device startup, which means it is not affected by the system's power supply voltage fluctuations.
2. Masked magnitude comparat

In [ ]:
#vstore.delete_collection() Let us completely delete the collection, thereby freeing the associated resources on Astra DB:

In [ ]:
response = query_engine.query("explain On-Chip System Connectivity {and Bridges}")
print(response)

Based on the new context provided, the original answer can be refined to better address the query. Here's a revised version of the answer:

On-Chip System Connectivity {and Bridges} refers to the various interfaces and bridges within the AURIX™ TC3xx Platform that enable communication between different components on the same chip. These interfaces and bridges allow different parts of the system to communicate with each other, enabling the efficient exchange of data and instructions.

The SFI_F2S Bridge is one of the key interfaces and bridges in the AURIX™ TC3xx Platform, which connects the System Peripheral Bus (SPB) to the SRI Fabric. This bridge implements a uni-directional bus bridge that forwards transactions from an implementation of an FPI protocol bus, such as the SPB, to the SRI Fabric. The bridge supports all FPI transactions on the SPB and those transactions of the SRI Fabric required to implement them.

The SFI_F2S Bridge is transparent for the address of a transaction and 